In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
from dbfread import DBF
import folium


In [13]:

base = Path('society-maps/DHA')
dbf_path = base / 'Merged_PCT_Vectorized.dbf'
if not dbf_path.exists():
    raise FileNotFoundError(f"DBF not found: {dbf_path}")

# Load DBF attributes (no geometry)
records = list(DBF(str(dbf_path), load=True))
dha = pd.DataFrame(records)
print(f'Loaded DBF attributes only (no geometry): {dbf_path.name}')

# Quick preview
print(dha.shape)
dha.head(3)

Loaded DBF attributes only (no geometry): Merged_PCT_Vectorized.dbf
(1809807, 2)


,DN,Area
0,8,57.853
1,3,4.132
2,3,8.265


In [18]:


split_base = Path("society-maps/DHA/Split_Areas")
shp_path = split_base / "DHA_AREA_split.shp"

# Read shapefile
gdf = gpd.read_file(shp_path)

# Optional: inspect the data
print(gdf.head())
print(gdf.crs)  # Check projection

# If CRS is not WGS84, convert so folium works properly
if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
    gdf = gdf.to_crs(epsg=4326)

# Save to GeoJSON
geojson_path = split_base / "DHA_AREA_split.geojson"
gdf.to_file(geojson_path, driver="GeoJSON")
print(f"GeoJSON saved to: {geojson_path}")

# Create folium map centered on the data
m = folium.Map(location=[gdf.geometry.centroid.y.mean(),
                         gdf.geometry.centroid.x.mean()],
               zoom_start=14)

gdf["Name"] = gdf["id"].astype(str).map(id_mappings) if gdf["id"].astype(str).map(id_mappings).notna().any() else gdf["id"].astype(str)

# Add the GeoJSON layer
folium.GeoJson(
    gdf,
    name="DHA Areas",
    popup=folium.GeoJsonPopup(fields=["Name"], aliases=["Area ID:"])
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save map to HTML
map_path = split_base / "dha_map.html"
m.save(map_path)
print(f"Map saved to: {map_path}")


   id                                           geometry
0   1  POLYGON ((74.35763 31.4775, 74.38137 31.48744,...
1   2  POLYGON ((74.38145 31.48739, 74.38211 31.48971...
2   3  POLYGON ((74.36678 31.46315, 74.38894 31.4703,...
3   4  POLYGON ((74.39345 31.4538, 74.39259 31.46683,...
4   5  POLYGON ((74.44678 31.45479, 74.45226 31.45042...
EPSG:4326
GeoJSON saved to: society-maps\DHA\Split_Areas\DHA_AREA_split.geojson
Map saved to: society-maps\DHA\Split_Areas\dha_map.html
Map saved to: society-maps\DHA\Split_Areas\dha_map.html


   id                                           geometry
0   1  POLYGON ((74.35763 31.4775, 74.38137 31.48744,...
1   2  POLYGON ((74.38145 31.48739, 74.38211 31.48971...
2   3  POLYGON ((74.36678 31.46315, 74.38894 31.4703,...
3   4  POLYGON ((74.39345 31.4538, 74.39259 31.46683,...
4   5  POLYGON ((74.44678 31.45479, 74.45226 31.45042...
EPSG:4326
GeoJSON saved to: society-maps\DHA\Split_Areas\DHA_AREA_split.geojson
Map saved to: society-maps\DHA\Split_Areas\dha_map.html
Map saved to: society-maps\DHA\Split_Areas\dha_map.html


C:\Users\Ahmad Hassan\AppData\Local\Temp\ipykernel_45556\1462208590.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[gdf.geometry.centroid.y.mean(),
C:\Users\Ahmad Hassan\AppData\Local\Temp\ipykernel_45556\1462208590.py:30: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf.geometry.centroid.x.mean()],


## Folium Mapping for DHA

In [7]:
id_mappings = {
    "1": "DHA Phase 3",
    "2": "DHA Phase 1,2",
    "3": "DHA Phase 4",
    "4": "DHA Phase 5",
    "5": "DHA Phase 9 Town",
    "6": "Askari 11",
    "7": "DHA Phase 8 - Ex Park View",
    "8": "DHA Phase 8 - Ex Air Avenue",
    "9": "DHA Phase 8",
    "10": "DHA Phase 8",
    "11": "DHA Phase 6",
    "12": "DHA Phase 7",
    "13": "DHA Phase 7",
    "14": "DHA Phase 7",
}

In [14]:
import re

# Load data
listings = pd.read_csv("./data/zameen_lahore_data.csv")
polys = gpd.read_file("society-maps/DHA/Split_Areas/DHA_AREA_split.geojson")

# Ensure polygons are in WGS84 for folium
if polys.crs and polys.crs.to_string() != "EPSG:4326":
    polys = polys.to_crs(epsg=4326)

# Map polygon IDs to human-friendly names using provided mapping
polys["Group"] = polys["id"].astype(str).map(id_mappings)

# Build normalized match list from mapping values
# We'll create a mapping from canonical group name -> list of partial keywords
canonical_groups = sorted(set([v for v in id_mappings.values() if isinstance(v, str)]))
# Expand a few common variants for robustness
keyword_expansions = {
    "DHA Phase 1,2": ["dha phase 1", "dha phase 2", "dha phase 1-2", "dha phase 1 , 2"],
    "DHA Phase 9 Town": ["dha phase 9 town", "phase 9 town"],
    "DHA Phase 8 - Ex Park View": ["dha phase 8 ex park view", "ex park view", "park view"],
    "DHA Phase 8 - Ex Air Avenue": ["dha phase 8 ex air avenue", "ex air avenue", "air avenue"],
}

# Build search patterns per canonical group (lowercased)
search_patterns = {}
for name in canonical_groups:
    base = [name.lower()]
    base += keyword_expansions.get(name, [])
    # also add simple "DHA Phase X" variant tokens to catch usual text
    m = re.findall(r"phase\s*\d+", name.lower())
    base += [f"dha {t}" for t in m]
    # dedupe
    tokens = sorted(set([t.strip() for t in base if t.strip()]))
    search_patterns[name] = tokens

# Helper to parse price strings to PKR
mults = {"crore": 10_000_000, "lac": 100_000, "lakh": 100_000}

def parse_price(price_str):
    if price_str is None or (hasattr(price_str, '__len__') and len(str(price_str).strip()) == 0):
        return None
    s = str(price_str).strip().lower()
    if s == 'nan' or s == '':
        return None
    m = re.search(r"([\d\.]+)", s)
    if not m:
        return None
    val = float(m.group(1))
    for k, mult in mults.items():
        if k in s:
            return val * mult
    return val

if "Price" not in listings.columns:
    raise KeyError("Expected column 'Price' in listings")

listings["Price_PKR"] = listings["Price"].apply(parse_price)

# Choose location column
loc_col = "Location" if "Location" in listings.columns else ("location" if "location" in listings.columns else None)
if not loc_col:
    raise KeyError("Could not find a Location column in listings")

# Assign group by partial match
listings["Group"] = None
loc_series = listings[loc_col].fillna("").astype(str)
for canon, tokens in search_patterns.items():
    # vectorized OR of substring contains
    mask = pd.Series(False, index=listings.index)
    for tok in tokens:
        mask |= loc_series.str.lower().str.contains(re.escape(tok), na=False)
    listings.loc[mask & listings["Group"].isna(), "Group"] = canon

# Compute average price per group
avg_prices = (
    listings.dropna(subset=["Group"]).groupby("Group", as_index=False)["Price_PKR"].mean()
)
avg_prices.rename(columns={"Price_PKR": "AvgPrice"}, inplace=True)

# Join to polygons and build heatmap
polys2 = polys.dropna(subset=["Group"]).merge(avg_prices, on="Group", how="left")

# Fallback label if AvgPrice missing
# Format prices for display and fill NaN values
def format_price(price):
    if pd.isna(price) or price == 0:
        return "No Data"
    if price >= 10_000_000:  # 1 crore or more
        return f"{price/10_000_000:.2f} Crore"
    elif price >= 100_000:  # 1 lac or more
        return f"{price/100_000:.2f} Lac"
    else:
        return f"{price:,.0f} PKR"

polys2["AvgPrice"] = polys2["AvgPrice"].fillna(0)
polys2["FormattedPrice"] = polys2["AvgPrice"].apply(format_price)

m = folium.Map(location=[31.4700, 74.4120], zoom_start=12, tiles="cartodbpositron")

ch = folium.Choropleth(
    geo_data=polys2.to_json(),
    data=polys2,
    columns=["Group", "AvgPrice"],
    key_on="feature.properties.Group",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Average Listing Price (PKR)",
).add_to(m)

folium.GeoJsonTooltip(
    fields=["Group", "FormattedPrice"],
    aliases=["Area", "Avg Price (PKR)"],
    localize=True,
    sticky=True,
).add_to(ch.geojson)

m.save("dha_price_heatmap.html")
print("Saved heatmap to dha_price_heatmap.html")

# Also show several partial-match approaches for reference
print("Examples of partial match checks:")
example_loc = "10 Marla House in DHA Phase 8 - Ex Air Avenue"
print("- substring in string:", "DHA Phase 8".lower() in example_loc.lower())
print("- pandas contains:", pd.Series([example_loc]).str.contains("phase\\s*8", case=False, regex=True).iloc[0])
print("- any token:", any(tok in example_loc.lower() for tok in search_patterns["DHA Phase 8 - Ex Air Avenue"]))

Saved heatmap to dha_price_heatmap.html
Examples of partial match checks:
- substring in string: True
- pandas contains: True
- any token: True


## Extraction of Lahore Boundaries

In [20]:
import geopandas as gpd
from pathlib import Path

# Define the paths to the uploaded files
district_file = Path("geo-data/lahore-boundary/District Punjab.geojson")
tehsil_file = Path("geo-data/lahore-boundary/Tehsil Punjab.geojson")

districts_gdf = gpd.read_file(district_file)

lahore_district = districts_gdf[districts_gdf['DistrictCo'] == 'Lahore']


lahore_boundary = lahore_district.dissolve()
output_lahore_boundary_file = Path("Lahore_Boundary.geojson")
lahore_boundary.to_file(output_lahore_boundary_file, driver='GeoJSON')
print(f"Successfully created the unified Lahore boundary file: {output_lahore_boundary_file}")




tehsils_gdf = gpd.read_file(tehsil_file)
lahore_tehsils = tehsils_gdf[tehsils_gdf['DistrictCo'] == 'Lahore']


output_lahore_tehsils_file = Path("Lahore_Tehsils.geojson")
lahore_tehsils.to_file(output_lahore_tehsils_file, driver='GeoJSON')

print(f"Successfully created the Lahore tehsils file: {output_lahore_tehsils_file}")
print(f"Found {len(lahore_tehsils)} tehsils in Lahore.")

# Assuming 'TEHSIL_N' or 'Tehsil_Nam' is the correct column for Tehsil names based on inspection
# If this next line gives an error, find the right tehsil name column in the printed list above.
if 'TehsilName' in lahore_tehsils.columns:
    print("Tehsils found:", list(lahore_tehsils['TehsilName']))
elif 'Tehsil_Nam' in lahore_tehsils.columns:
     print("Tehsils found:", list(lahore_tehsils['Tehsil_Nam']))
else:
    print("Could not determine the Tehsil name column. Please inspect the printed column list.")


Successfully created the unified Lahore boundary file: Lahore_Boundary.geojson
Successfully created the Lahore tehsils file: Lahore_Tehsils.geojson
Found 0 tehsils in Lahore.
Tehsils found: []
